In [1]:
!pip install -q pyspark


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pyspark

In [3]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/05/16 07:20:32 WARN Utils: Your hostname, DESKTOP-685S3TT resolves to a loopback address: 127.0.1.1; using 172.20.241.155 instead (on interface eth0)
23/05/16 07:20:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/16 07:20:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


2 Data Structures:

1. Resiliant Distributed Dataset (RDD)
  - Low level
  - List item
  - Efficient but not as convenient as DFs
  -Immutable
  -Processed with Map, Filter, or Reduce
  -Typically used to process linear data
  - More difficult to use than DFs
2. DataFrame

In [4]:
text = 'the quick brown fox jumped over the lazy dogs'
lst = text.split()

print(type(text), text)
print(type(lst), lst)

<class 'str'> the quick brown fox jumped over the lazy dogs
<class 'list'> ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dogs']


In [5]:
text_rdd = spark.sparkContext.parallelize(text)
print(type(text_rdd), text_rdd)
print(type(text_rdd.collect()))

<class 'pyspark.rdd.RDD'> ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287


<class 'list'>


In [6]:
lst_rdd = spark.sparkContext.parallelize(lst)
print(type(lst_rdd), lst_rdd)
print(type(lst_rdd.collect()))

<class 'pyspark.rdd.RDD'> ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:287
<class 'list'>


In [8]:
rdd = spark.sparkContext.textFile('/home/blair/cognixia/AdvancedPhase/Linux_Workspace/Linux_Workspace/dirWithFiles/beemovie.txt')
rdd.take(10)

['According to all known laws of aviation, there is no way a bee should be able to fly.',
 'Its wings are too small to get its fat little body off the ground.',
 "The bee, of course, flies anyway because bees don't care what humans think is impossible.",
 'Yellow, black. Yellow, black. Yellow, black. Yellow, black.',
 'Ooh, black and yellow!',
 "Let's shake it up a little.",
 'Barry! Breakfast is ready!',
 'Coming!',
 'Hang on a second.',
 'Hello?']

In [9]:
rdd.count() #returns the total number of lines

1363

In [10]:
rdd.distinct().count() #returns distinct number of lines

1302

In [11]:
rdd.filter(lambda line: 'bee' in line.lower()).count()

146

In [13]:
words = rdd.flatMap(lambda line: line.split()) \
            .map(lambda word: word.lower()) \
            .flatMap(lambda word: word.replace(".","").replace('"',"").replace('!',"").replace("?","").split("-")) \
            .flatMap(lambda word: word.split(","))
            
print(words.count(), words.collect())

9791 ['according', 'to', 'all', 'known', 'laws', 'of', 'aviation', '', 'there', 'is', 'no', 'way', 'a', 'bee', 'should', 'be', 'able', 'to', 'fly', 'its', 'wings', 'are', 'too', 'small', 'to', 'get', 'its', 'fat', 'little', 'body', 'off', 'the', 'ground', 'the', 'bee', '', 'of', 'course', '', 'flies', 'anyway', 'because', 'bees', "don't", 'care', 'what', 'humans', 'think', 'is', 'impossible', 'yellow', '', 'black', 'yellow', '', 'black', 'yellow', '', 'black', 'yellow', '', 'black', 'ooh', '', 'black', 'and', 'yellow', "let's", 'shake', 'it', 'up', 'a', 'little', 'barry', 'breakfast', 'is', 'ready', 'coming', 'hang', 'on', 'a', 'second', 'hello', 'barry', 'adam', 'can', 'you', 'believe', 'this', 'is', 'happening', 'i', "can't", "i'll", 'pick', 'you', 'up', 'looking', 'sharp', 'use', 'the', 'stairs', '', 'your', 'father', 'paid', 'good', 'money', 'for', 'those', 'sorry', "i'm", 'excited', "here's", 'the', 'graduate', "we're", 'very', 'proud', 'of', 'you', '', 'son', 'a', 'perfect', 'rep

In [15]:
words.distinct().count()

1850

In [16]:
lengths = words.map(lambda word: len(word))
print(lengths.collect())
print(lengths.max())
print(lengths.min())
print(lengths.mean())
print(lengths.sum())

[9, 2, 3, 5, 4, 2, 8, 0, 5, 2, 2, 3, 1, 3, 6, 2, 4, 2, 3, 3, 5, 3, 3, 5, 2, 3, 3, 3, 6, 4, 3, 3, 6, 3, 3, 0, 2, 6, 0, 5, 6, 7, 4, 5, 4, 4, 6, 5, 2, 10, 6, 0, 5, 6, 0, 5, 6, 0, 5, 6, 0, 5, 3, 0, 5, 3, 6, 5, 5, 2, 2, 1, 6, 5, 9, 2, 5, 6, 4, 2, 1, 6, 5, 5, 4, 3, 3, 7, 4, 2, 9, 1, 5, 4, 4, 3, 2, 7, 5, 3, 3, 6, 0, 4, 6, 4, 4, 5, 3, 5, 5, 3, 7, 6, 3, 8, 5, 4, 5, 2, 3, 0, 3, 1, 7, 6, 4, 0, 3, 3, 4, 5, 2, 1, 3, 1, 5, 5, 4, 3, 3, 4, 2, 4, 4, 2, 6, 2, 4, 2, 2, 5, 2, 2, 3, 3, 3, 3, 5, 0, 1, 4, 3, 0, 4, 6, 2, 3, 5, 3, 0, 4, 3, 0, 5, 2, 4, 4, 3, 1, 6, 7, 3, 0, 10, 5, 7, 3, 4, 2, 5, 4, 5, 6, 0, 5, 4, 4, 6, 5, 4, 7, 5, 4, 7, 3, 4, 1, 4, 1, 3, 3, 10, 6, 3, 4, 3, 3, 4, 4, 9, 2, 0, 5, 5, 0, 7, 1, 8, 5, 4, 4, 5, 7, 4, 3, 5, 2, 3, 7, 2, 0, 3, 3, 5, 9, 5, 0, 5, 7, 0, 3, 3, 5, 5, 2, 2, 1, 8, 4, 1, 7, 1, 5, 2, 5, 4, 4, 6, 3, 2, 3, 3, 1, 4, 4, 13, 2, 9, 4, 4, 3, 3, 6, 3, 2, 5, 4, 9, 3, 0, 5, 1, 3, 2, 4, 5, 3, 13, 4, 0, 4, 0, 5, 2, 3, 3, 2, 3, 3, 3, 4, 10, 8, 0, 7, 0, 13, 4, 0, 6, 7, 4, 8, 7, 0, 3, 4, 4, 10, 5

15


0


3.846593810642424


37662


In [17]:
print(words.max())
print(words.max(len))

yowser


congratulations


In [18]:
# reduce
words.reduce(lambda current, word: word if len(word) > len(current) else current)

'congratulations'

In [14]:
word_data = words.map(lambda word: (word, len(word)))
word_data.collect()

[('according', 9),
 ('to', 2),
 ('all', 3),
 ('known', 5),
 ('laws', 4),
 ('of', 2),
 ('aviation', 8),
 ('', 0),
 ('there', 5),
 ('is', 2),
 ('no', 2),
 ('way', 3),
 ('a', 1),
 ('bee', 3),
 ('should', 6),
 ('be', 2),
 ('able', 4),
 ('to', 2),
 ('fly', 3),
 ('its', 3),
 ('wings', 5),
 ('are', 3),
 ('too', 3),
 ('small', 5),
 ('to', 2),
 ('get', 3),
 ('its', 3),
 ('fat', 3),
 ('little', 6),
 ('body', 4),
 ('off', 3),
 ('the', 3),
 ('ground', 6),
 ('the', 3),
 ('bee', 3),
 ('', 0),
 ('of', 2),
 ('course', 6),
 ('', 0),
 ('flies', 5),
 ('anyway', 6),
 ('because', 7),
 ('bees', 4),
 ("don't", 5),
 ('care', 4),
 ('what', 4),
 ('humans', 6),
 ('think', 5),
 ('is', 2),
 ('impossible', 10),
 ('yellow', 6),
 ('', 0),
 ('black', 5),
 ('yellow', 6),
 ('', 0),
 ('black', 5),
 ('yellow', 6),
 ('', 0),
 ('black', 5),
 ('yellow', 6),
 ('', 0),
 ('black', 5),
 ('ooh', 3),
 ('', 0),
 ('black', 5),
 ('and', 3),
 ('yellow', 6),
 ("let's", 5),
 ('shake', 5),
 ('it', 2),
 ('up', 2),
 ('a', 1),
 ('little', 6

In [38]:
stop_words = ['', 'and', 'a', 'of', 'is', 'it', 'in', 'to', 'the', 'for', 'I', 'this', 'that', 'had', 'all', 'your', 'I\'m', 'no', 'don\'t', 'out', 'i\'m', 'i', 'we', 'are', 'on', 'what', 'have', 'you', 'not', 'me', 'my', 'just', 'it\'s', 'do', 'be', 'with', 'they', 'was', 'you\'re', 'that\'s', 'here', 'right']

In [39]:
words.filter(lambda word: word not in stop_words) \
    .map(lambda word: (word, 1)) \
        .groupByKey().mapValues(sum) \
            .sortBy(lambda row: row[1], ascending=False) \
                .take(10)
    # .reduceByKey(lambda curr, next: curr + next) ***same as .groupByKey().mapValues(sum)***

[('bee', 93),
 ('bees', 57),
 ('know', 53),
 ('barry', 50),
 ('honey', 49),
 ('like', 39),
 ('get', 37),
 ('but', 36),
 ('our', 36),
 ('so', 36)]